# Step 5 - Analysis of bicycle network results
## Project: Algorithmic bicycle network design
#### Michael Szell, Tyler Perlman, Sayat Mimar, Gourab Ghoshal, Roberta Sinatra

This notebook takes the existing infrastructure, the results from 03_poi_based_generation and 04_connect_clusters and calculates/analyzes a number of measures:
* cost (length)
* coverage  
* directness  
* efficiency

Contact: Michael Szell (michael.szell@gmail.com)  
Created: 2020-07-08  
Last modified: 2020-07-21

## Preliminaries

### Parameters

In [ ]:
debug = False # If True, will produce plots and/or verbose output to double-check
%run -i "../parameters/parameters.py"

### Setup

In [ ]:
%run -i path.py
%run -i setup.py

%matplotlib inline
%load_ext watermark
%watermark -n -v -m -g -iv

### Functions

In [ ]:
%run -i functions.py

## Analyze existing infrastructure

In [ ]:
for placeid, placeinfo in cities.items():
    print(placeid + ": Analying existing infrastructure.")
    
    # output_place is one static file for the existing city. This can be compared to the generated infrastructure.
    # Make a check if this file was already generated - it only needs to be done once. If not, generate it:
    filename = placeid + "_existing.csv"
    if not os.path.isfile(PATH["results"] + placeid + "/" + filename):
        empty_metrics = {
                         "length":0,
                         "coverage": 0,
                         "directness": 0,
                         "poi_coverage": 0,
                         "components": 0,
                         "efficiency_global": 0,
                         "efficiency_local": 0
                        }
        output_place = {}
        for networktype in networktypes:
            output_place[networktype] = copy.deepcopy(empty_metrics)

        # Analyze all networks     
        Gs = {}
        for networktype in networktypes:
            Gs[networktype] = csv_to_ig(PATH["data"] + placeid + "/", placeid, networktype)
            Gs[networktype + "_simplified"] = csv_to_ig(PATH["data"] + placeid + "/", placeid, networktype + "_simplified")
        
        with open(PATH["data"] + placeid + "/" + placeid + '_poi_railwaystation_nnidscarall.csv') as f:
            nnids = [int(line.rstrip()) for line in f]

        for networktype in networktypes:
            if debug: print(placeid + ": Analying results: " + networktype)
            # To do: Call calculate_metrics for existing infrastructure, passing ox-simplified graphs as GT_abstract
            metrics = calculate_metrics(Gs[networktype], Gs[networktype + "_simplified"], Gs['carall'], nnids, buffer_walk, numnodepairs, True)
            for key, val in metrics.items():
                output_place[networktype][key] = val
        
        # Write to CSV
        # https://stackoverflow.com/questions/29400631/python-writing-nested-dictionary-to-csv
        with open(PATH["results"] + placeid + "/" + filename, 'w') as f:
            fields = ['network', 'components', 'coverage', 'directness', 'length', 'poi_coverage', 'efficiency_global', 'efficiency_local']
            w = csv.DictWriter(f, fields)
            w.writeheader()
            for key, val in sorted(output_place.items()):
                row = {'network': key}
                row.update(val)
                w.writerow(row)
        # TO DO: TEST

## Analyze POI based results

In [ ]:
for placeid, placeinfo in cities.items():
    print(placeid + ": Analying results")

    # Load networks
    G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall')
    
    # Load POIs
    with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
        nnids = [int(line.rstrip()) for line in f]
            
    # Load results
    filename = placeid + '_poi_' + poi_source + "_" + prune_measure
    resultfile = open(PATH["results"] + placeid + "/" + filename + ".pickle",'rb')
    res = pickle.load(resultfile)
    resultfile.close()
    if debug: pp.pprint(res)
        
        
    # output contains lists for all the prune_quantile values of the corresponding results
    output = {"length":[],
              "coverage": [],
              "directness": [],
              "poi_coverage": [],
              "components": [],
              "efficiency_global": [],
              "efficiency_local": []
             }

    for GT, GT_abstract, prune_quantile in zip(res["GTs"], res["GT_abstracts"], res["prune_quantiles"]):
        if debug: print(prune_quantile, len(GT.vs))

        metrics = calculate_metrics(GT, GT_abstract, G_carall, nnids, buffer_walk, numnodepairs)

        for key in output.keys():
            output[key].append(metrics[key])
        
    # Write to CSV
    filename = placeid + '_poi_' + poi_source + "_" + prune_measure + ".csv"

    with open(PATH["results"] + placeid + "/" + filename, 'w') as f:
        w = csv.writer(f)
        w.writerow(output.keys())
        w.writerows(zip(*output.values()))

In [ ]:
if debug:
    pp.pprint(output)
    
    fig = plt.figure(figsize=(4, 3)) # create figure object with a (width,height)
    axes = fig.add_axes([0, 0, 1, 1]) # left, bottom, width, height (range 0 to 1)
    for pol in cov: # cov is generally a MultiPolygon, so we iterate through its Polygons
        y, x = pol.exterior.coords.xy
        patch1 = matplotlib.patches.Polygon(np.stack((np.asarray(x), -1*np.asarray(y)), axis=-1))
        axes.add_patch(patch1)
    axes.plot()

## Analyze cluster based results

In [ ]:
# import warnings
# warnings.filterwarnings("ignore")
%run -i functions.py
for placeid, placeinfo in cities.items():
    print(placeid + ": Analyzing results")

    # Load networks
    G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall')
    
    # Load POIs
    with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
        nnids = [int(line.rstrip()) for line in f]
    
    # Load results
    filename = placeid + '_clusters_' + prune_measure + "_cutoff" + cutofftype + "{:.2f}".format(cutoff) + ".pickle"
    resultfile = open(PATH["results"] + placeid + "/" + filename,'rb')
    res = pickle.load(resultfile)
    resultfile.close()
    if debug: pp.pprint(res)
        
        
    # output contains lists for all the prune_quantile values of the corresponding results
    output = {"length":[],
              "coverage": [],
              "directness": [],
              "poi_coverage": [],
              "components": [],
              "efficiency_global": [],
              "efficiency_local": []
             }

    for GT, GT_abstract, prune_quantile in zip(res["GTs"], res["GT_abstracts"], res["prune_quantiles"]):
        if debug: print(prune_quantile, len(GT.vs))

        metrics = calculate_metrics(GT, GT_abstract, G_carall, nnids, buffer_walk, numnodepairs)

        for key in output.keys():
            output[key].append(metrics[key])
        
    # Write to CSV
    filename = placeid + '_clusters_' + prune_measure + "_cutoff" + cutofftype + "{:.2f}".format(cutoff) + ".csv"
    with open(PATH["results"] + placeid + "/" + filename, 'w') as f:
        w = csv.writer(f)
        w.writerow(output.keys())
        w.writerows(zip(*output.values()))

In [ ]:
Audio(sound_file, autoplay=True)